In [1]:
from __future__ import division
from numpy.random import randn
from pandas import Series
import numpy as np
import pandas as pd
from datetime import datetime
import math
np.set_printoptions(precision=4)
import sys; sys.path.append('book_scripts')
%pwd

u'E:\\project\\pychram\\book_scripts'

In [2]:
def load_stock_base_msg():
    stock_data_path = 'E:/project/pychram/traderesp/base/output-csv/'
    stock_data_cvs = stock_data_path + 'output-2013-2014-all-stock-base-msg.csv'
    #
    stat = pd.read_csv(stock_data_cvs,encoding='gbk',index_col=['code'])
    return stat

In [3]:
def get_date_period_staticstics(begtime,endtime):
    #
    stock_data_path = 'E:/project/pychram/traderesp/strategy/output-csv/'
    stock_data_cvs = stock_data_path + 'output-stock-history-adjust-m26-kdj-only-jx-up-buy-' \
                                     + begtime.strftime('%Y%m%d') + '-' + endtime.strftime('%Y%m%d') + '.csv'
    #
    stat = pd.read_csv(stock_data_cvs,parse_dates=['buy_date','sell_date'],encoding='gbk')
    
    #逐笔利润
    stat['profit'] = stat['sell_price'] - stat['buy_price']
    #
    action_num = len(stat)
    suc_num = stat[stat['profit'] > 0.00 ].count() 
    fail_num = action_num - suc_num
    #
    tmp_profit = stat.groupby(['code']).sum()
    year_action_num = len(tmp_profit)
    tmp_profit['profit_rate'] = (tmp_profit['sell_price'] - tmp_profit['buy_price']) / tmp_profit['buy_price']
    year_suc_num = tmp_profit[tmp_profit['profit_rate'] > 0.00].count()
    year_fail_num = year_action_num - year_suc_num
    #
    year_suc_num_gt_percet_5 = tmp_profit[tmp_profit['profit_rate'] > 0.05].count()
    #
    stat_sum_all = stat.sum()
    code_unique_num = len(stat['code'].unique())
    s_ge_b_num = stat[stat['sell_price'] > stat['buy_price']].count()
    #
    print '-----  ',begtime,'  ----  ',endtime,'  -----'
    print '股票总数(排重) :',year_action_num
    print '买卖总次数(一次买卖算一次) :',action_num
    print '其中获利次数 :',suc_num['profit']
    print '其中亏损次数 :',fail_num['profit']
    print '获利次数比率 :',suc_num['profit'] / action_num
    print '亏损次数比率 :',fail_num['profit'] / action_num
    print '年度获利股票数 :',year_suc_num['profit_rate']
    print '年度亏损股票数 :',year_fail_num['profit_rate']
    print '年度获利股票数比率 :',year_suc_num['profit_rate'] / year_action_num
    print '年度亏损股票数比率 :',year_fail_num['profit_rate'] / year_action_num
    print '年度获利股票数( 盈利大于5%) :',year_suc_num_gt_percet_5['profit_rate']
    print '年度获利股票数比率( 盈利大于5%) :',year_suc_num_gt_percet_5['profit_rate'] / year_action_num
    print '---------------------------------------------------------------------'
    #
    return tmp_profit[tmp_profit['profit_rate'] > 0.00].sort_values('profit_rate',ascending=False),tmp_profit,stat

In [4]:
def show_date_staticstics():
    code1,code1_tmp1,code1_tmp2 = get_date_period_staticstics(datetime(2013,1,1),datetime(2013,12,31))
    code2,code2_tmp1,code2_tmp2 = get_date_period_staticstics(datetime(2014,1,1),datetime(2014,12,31))
    code3 = set(code1.index) & set(code2.index)
    year_2_num = len(code3)
    print '连续2年上涨股票数(获利股票数) :',year_2_num
    print '连续2年上涨股票数比率 :',year_2_num / len(code1) if len(code1) > 0 else 0
    #--------------------------------------------------------
    code1_gt_percent_5 =  code1[code1['profit_rate'] > 0.05]
    code2_gt_percent_5 =  code2[code2['profit_rate'] > 0.05]
    code3_gt_percent_5 =  set(code1_gt_percent_5.index) & set(code2_gt_percent_5.index)
    print '连续2年上涨股票数(获利股票数 大于 5%) :',len(code3_gt_percent_5)
    print '连续2年上涨股票数比率(获利大于 5%) :',len(code3_gt_percent_5) / len(code1) if len(code1) > 0 else 0
    #--------------------------------------------------------
    code4 = code1.index[0:100]
    code5 = code2.index[0:100]
    code6 = set(code4) & set(code5)
    year_2_num_from_100 = len(code6)
    print '连续2年上涨股票数(Top 100) :',year_2_num_from_100
    code7 = code1.index[0:200]
    code8 = code2.index[0:200]
    code9 = set(code7) & set(code8)
    year_2_num_from_200 = len(code9)
    print '连续2年上涨股票数(Top 200) :',year_2_num_from_200
    code10 = code1.index[0:300]
    code11 = code2.index[0:300]
    code12 = set(code10) & set(code11)
    year_2_num_from_300 = len(code12)
    print '连续2年上涨股票数(Top 300) :',year_2_num_from_300
    code13 = code1.index[0:400]
    code14 = code2.index[0:400]
    code15 = set(code13) & set(code14)
    year_2_num_from_400 = len(code15)
    print '连续2年上涨股票数(Top 400) :',year_2_num_from_400
    code16 = code1.index[0:500]
    code17 = code2.index[0:500]
    code18 = set(code16) & set(code17)
    year_2_num_from_500 = len(code18)
    print '连续2年上涨股票数(Top 500) :',year_2_num_from_500
    #
    code_bing = pd.merge(code1,code2,left_index=True, right_index=True)
    return code_bing,code_bing[(code_bing['profit_rate_x'] > 0.05) & (code_bing['profit_rate_y'] > 0.05)]

In [5]:
code_base_msg = load_stock_base_msg()
code_profit_rate_ge_0, code_profit_rate_ge_5 = show_date_staticstics()
code_profit_rate_ge_5 = code_profit_rate_ge_5[['profit_rate_x','profit_rate_y']]

-----   2013-01-01 00:00:00   ----   2013-12-31 00:00:00   -----
股票总数(排重) : 2468
买卖总次数(一次买卖算一次) : 17547
其中获利次数 : 6842
其中亏损次数 : 10705
获利次数比率 : 0.389924203568
亏损次数比率 : 0.610075796432
年度获利股票数 : 1343
年度亏损股票数 : 1125
年度获利股票数比率 : 0.544165316045
年度亏损股票数比率 : 0.455834683955
年度获利股票数( 盈利大于5%) : 209
年度获利股票数比率( 盈利大于5%) : 0.0846839546191
---------------------------------------------------------------------
-----   2014-01-01 00:00:00   ----   2014-12-31 00:00:00   -----
股票总数(排重) : 2574
买卖总次数(一次买卖算一次) : 23256
其中获利次数 : 11346
其中亏损次数 : 11910
获利次数比率 : 0.487874097007
亏损次数比率 : 0.512125902993
年度获利股票数 : 1642
年度亏损股票数 : 932
年度获利股票数比率 : 0.637917637918
年度亏损股票数比率 : 0.362082362082
年度获利股票数( 盈利大于5%) : 391
年度获利股票数比率( 盈利大于5%) : 0.151903651904
---------------------------------------------------------------------
连续2年上涨股票数(获利股票数) : 836
连续2年上涨股票数比率 : 0.622486969471
连续2年上涨股票数(获利股票数 大于 5%) : 17
连续2年上涨股票数比率(获利大于 5%) : 0.0126582278481
连续2年上涨股票数(Top 100) : 2
连续2年上涨股票数(Top 200) : 8
连续2年上涨股票数(Top 300) : 23
连续2年上涨股票数(Top 400) : 3

In [6]:
code_bing = pd.merge(code_base_msg,code_profit_rate_ge_5,left_index=True, right_index=True)

In [7]:
code_bing.columns

Index([    u'code_name',       u'sina_hy',       u'sina_gn',       u'sina_dy',
       u'profit_rate_x', u'profit_rate_y'],
      dtype='object')

In [8]:
def trim_float_num(x):
    #print type(x)
    #print 'x=',x
    xx = 0.000
    if x is None:
        xx = 0.000
    else:
        xx = float(str(x))
    # print xx
    if xx < 0.001:
        return "0.1 %"
    else:
        ret = '%.1f' %(xx * 100.00)
        return ret + ' %'

In [9]:
trim_profit_rate_x = code_bing['profit_rate_x'].map(trim_float_num)
code_bing['profit_rate_x'] = trim_profit_rate_x
trim_profit_rate_y = code_bing['profit_rate_y'].map(trim_float_num)
code_bing['profit_rate_y'] = trim_profit_rate_y

In [10]:
code_bing

,code_name,sina_hy,sina_gn,sina_dy,profit_rate_x,profit_rate_y
code,,,,,,
300288,朗玛信息,电子信息；电子信息,重组概念；4G概念；网络游戏；4G概念；重组概念,贵州,5.0 %,18.4 %
8,神州高铁,综合行业；综合行业,铁路基建；涉矿概念；业绩预升；涉矿概念；深圳本地；铁路基建,广东；深圳,5.8 %,14.6 %
300085,银之杰,电子信息；电子信息,重组概念；O2O模式；O2O模式；重组概念；业绩预降；股权激励,广东；深圳,12.4 %,14.5 %
600639,浦东金桥,开发区；开发区,含B股；上海自贸；金融参股；信托重仓；含B股；上海本地；参股金融；迪士尼,上海,9.7 %,11.1 %
601989,中国重工,船舶制造；船舶制造,军工航天；融资融券；海工装备；智能交通；智能交通；海工装备；保险重仓；券商重仓；融资融券；央...,北京,5.0 %,8.5 %
600663,陆家嘴,房地产；房地产,含B股；保障房；国企改革；上海自贸；保障房；信托重仓；含B股；上海本地；迪士尼,上海,10.0 %,8.0 %
300220,金运激光,电子器件；电子器件,武汉规划；3D打印；O2O模式；机器人概念；O2O模式；3D打印；武汉规划,湖北,5.4 %,7.9 %
600485,信威集团,电子信息；电子信息,IPV6概念；4G概念；4G概念；IPV6概念,北京,12.8 %,7.9 %
600555,九龙山,酒店旅游；酒店旅游,含B股；土地流转；赛马概念；赛马概念；土地流转；含B股；上海本地,上海,8.2 %,7.7 %


In [11]:
code1,code_2,code_all = get_date_period_staticstics(datetime(2013,1,1),datetime(2013,12,31))

-----   2013-01-01 00:00:00   ----   2013-12-31 00:00:00   -----
股票总数(排重) : 2468
买卖总次数(一次买卖算一次) : 17547
其中获利次数 : 6842
其中亏损次数 : 10705
获利次数比率 : 0.389924203568
亏损次数比率 : 0.610075796432
年度获利股票数 : 1343
年度亏损股票数 : 1125
年度获利股票数比率 : 0.544165316045
年度亏损股票数比率 : 0.455834683955
年度获利股票数( 盈利大于5%) : 209
年度获利股票数比率( 盈利大于5%) : 0.0846839546191
---------------------------------------------------------------------


In [12]:
code_all[code_all['code']==858]

,buy_date,buy_money,buy_num,buy_price,code,sell_date,sell_money,sell_num,sell_price,profit
263,2013-02-18,37864.797560,100,378.647976,858,2013-02-22,38721.288375,100,387.212884,8.564908
3253,2013-03-25,34559.160354,100,345.591604,858,2013-03-27,33507.344999,100,335.073450,-10.518154
5515,2013-05-07,34363.793698,100,343.637937,858,2013-06-07,34589.139512,100,345.891395,2.253458
7898,2013-07-17,31320.121269,100,313.201213,858,2013-07-19,30043.659423,100,300.436594,-12.764618
11733,2013-08-02,29965.847459,100,299.658475,858,2013-10-18,28066.689769,100,280.666898,-18.991577
13950,2013-10-22,28237.928886,100,282.379289,858,2013-10-24,27786.500579,100,277.865006,-4.514283
15435,2013-11-19,26152.013448,100,261.520134,858,2013-12-17,25000.012524,100,250.000125,-11.520009


In [13]:
code2,code21,code2_all = get_date_period_staticstics(datetime(2014,1,1),datetime(2014,12,31))

-----   2014-01-01 00:00:00   ----   2014-12-31 00:00:00   -----
股票总数(排重) : 2574
买卖总次数(一次买卖算一次) : 23256
其中获利次数 : 11346
其中亏损次数 : 11910
获利次数比率 : 0.487874097007
亏损次数比率 : 0.512125902993
年度获利股票数 : 1642
年度亏损股票数 : 932
年度获利股票数比率 : 0.637917637918
年度亏损股票数比率 : 0.362082362082
年度获利股票数( 盈利大于5%) : 391
年度获利股票数比率( 盈利大于5%) : 0.151903651904
---------------------------------------------------------------------


In [14]:
code2_all[code2_all['code']==858]

,buy_date,buy_money,buy_num,buy_price,code,sell_date,sell_money,sell_num,sell_price,profit
308,2014-02-13,23381.133728,100,233.811337,858,2014-02-26,23505.669612,100,235.056696,1.245359
5457,2014-04-03,26759.129144,100,267.591291,858,2014-04-22,26603.453464,100,266.034535,-1.556757
8046,2014-05-27,25871.872212,100,258.718722,858,2014-05-28,26214.338184,100,262.143382,3.424660
9847,2014-05-29,26136.507814,100,261.365078,858,2014-06-05,26198.762473,100,261.987625,0.622547
11881,2014-06-06,26385.559649,100,263.855596,858,2014-06-20,26525.653681,100,265.256537,1.400940
14132,2014-07-01,27568.631032,100,275.686310,858,2014-07-10,28719.339690,100,287.193397,11.507087
16385,2014-07-15,31134.213962,100,311.342140,858,2014-08-29,30356.271578,100,303.562716,-7.779424
18680,2014-09-03,31490.769961,100,314.907700,858,2014-10-22,28395.059918,100,283.950599,-30.957100
21027,2014-10-30,28070.920751,100,280.709208,858,2014-12-31,34845.539180,100,348.455392,67.746184


In [15]:
code2_all[code2_all['code']==858].sum()

buy_money     246798.738253
buy_num          900.000000
buy_price       2467.987383
code            7722.000000
sell_money    251364.087779
sell_num         900.000000
sell_price      2513.640878
profit            45.653495
dtype: float64

In [ ]:
code21.ix[858]